# Day 12: Wash Sale Rules

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class4_taxes_portfolio/week3_tax_optimization/day12_wash_sale_strategies.ipynb)

## Learning Objectives

By the end of this lesson, you will be able to:

1. **Understand Wash Sale Rules**: Explain the 61-day wash sale window and substantially identical securities
2. **Identify Wash Sale Violations**: Recognize transactions that trigger wash sale rules
3. **Calculate Cost Basis Adjustments**: Determine how disallowed losses adjust future cost basis
4. **Implement Legal Swap Strategies**: Use alternative securities to maintain market exposure
5. **Build Wash Sale Tracking**: Create Python tools to monitor and avoid wash sales

## 1. What is a Wash Sale?

A wash sale occurs when you sell a security at a loss and repurchase the same or "substantially identical" security within 30 days before or after the sale.

### The 61-Day Window

```
WASH SALE WINDOW (61 Days Total)
═════════════════════════════════════════════════════════════

      30 days before    SALE DAY    30 days after
    ◄───────────────────►│◄───────────────────►
    
    Day -30 ... Day -1   Day 0   Day 1 ... Day 30
    
    ┌─────────────────────────────────────────┐
    │  ANY purchase in this window triggers   │
    │  wash sale rule and disallows the loss  │
    └─────────────────────────────────────────┘
```

### Key Rule Components

1. **Sale at a Loss**: You must sell the security for less than your cost basis
2. **Repurchase Window**: Within 30 days before or 30 days after the sale (61 days total)
3. **Substantially Identical**: Same or very similar security
4. **Loss Disallowed**: You cannot claim the loss in the current year
5. **Basis Adjustment**: The disallowed loss is added to the cost basis of the replacement shares

## 2. Substantially Identical Securities

### What Counts as Substantially Identical?

```
SUBSTANTIALLY IDENTICAL (Triggers Wash Sale):
═══════════════════════════════════════════════
✗ Same stock (AAPL → AAPL)
✗ Same ETF (SPY → SPY)
✗ Call options on the same stock
✗ Convertible bonds → common stock
✗ Preferred stock → common stock (same company)

NOT Substantially Identical (Safe Swaps):
════════════════════════════════════════════
✓ Different companies (AAPL → MSFT)
✓ Different S&P 500 ETFs (SPY → VOO or IVV)
✓ Different total market ETFs (VTI → ITOT or SCHB)
✓ Stock → sector ETF (XLK technology ETF)
✓ Individual bond → bond ETF
✓ Mutual fund → similar ETF (different structure)
```

### Common Safe Swap Pairs

| Sell This | Buy This | Tracking Difference |
|-----------|----------|---------------------|
| SPY (S&P 500) | VOO or IVV | <0.01% |
| VTI (Total Market) | ITOT or SCHB | <0.02% |
| QQQ (Nasdaq-100) | QQQM | 0.00% (same holdings) |
| VEA (Developed Int'l) | IDEV or SCHF | <0.05% |
| VWO (Emerging Markets) | IEMG or SCHE | <0.10% |
| AGG (Total Bond) | BND or SCHZ | <0.02% |

## 3. How Wash Sales Affect Cost Basis

When a wash sale occurs, the loss is NOT permanently lost—it's deferred by adjusting the cost basis of the replacement shares.

### Cost Basis Adjustment Formula

```
New Cost Basis = Purchase Price + Disallowed Loss
```

### Example: Wash Sale Scenario

```
Step 1: Initial Purchase
═══════════════════════════════════════════
Date: Jan 1
Buy: 100 shares AAPL @ $150
Cost Basis: $15,000

Step 2: Sell at Loss
═══════════════════════════════════════════
Date: Dec 15
Sell: 100 shares AAPL @ $140
Sale Proceeds: $14,000
Loss: $1,000

Step 3: Repurchase (Wash Sale Violation!)
═══════════════════════════════════════════
Date: Dec 20 (5 days later - within 30 days!)
Buy: 100 shares AAPL @ $142
Purchase Price: $14,200

Step 4: Cost Basis Adjustment
═══════════════════════════════════════════
Disallowed Loss: $1,000
New Cost Basis: $14,200 + $1,000 = $15,200

Result:
  - Cannot deduct $1,000 loss this year
  - New shares have $15,200 basis (vs $14,200 paid)
  - Loss preserved for when you sell the new shares
```

## 4. Legal Swap Strategies

### Strategy 1: ETF Swaps

```
EXAMPLE: S&P 500 Index Swap
═══════════════════════════════════════════

Portfolio: Own SPY (SPDR S&P 500 ETF)
Loss: Down 15% ($10,000 → $8,500)

Action:
  1. Sell SPY for $8,500 loss
  2. Immediately buy VOO (Vanguard S&P 500 ETF)
  3. Harvest $1,500 loss
  4. Maintain S&P 500 exposure

Why This Works:
  - Different fund companies (State Street vs Vanguard)
  - Different ticker symbols
  - IRS considers them NOT substantially identical
  - Tracking difference is <0.01% (nearly identical returns)
```

### Strategy 2: Wait 31 Days

```
TIMELINE: 31-Day Waiting Period
═══════════════════════════════════════════

Day 0:  Sell AAPL at loss
Day 1-30: Wait (no AAPL purchases!)
Day 31: Safe to repurchase AAPL

Risk: Stock price may recover during wait
Solution: Use similar stock/ETF as temporary substitute
```

### Strategy 3: Sector ETF Substitute

```
EXAMPLE: Individual Stock → Sector ETF
═══════════════════════════════════════════

Portfolio: Own AAPL (down 20%)

Action:
  1. Sell AAPL at loss
  2. Buy XLK (Technology Select Sector ETF)
     - Contains AAPL as top holding (~20%)
     - Maintains tech sector exposure
  3. After 31 days, sell XLK and rebuy AAPL

Benefit: Maintain similar exposure during wait period
```

## 5. Python Implementation: Wash Sale Tracker

Let's build tools to track and avoid wash sales.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

print("Libraries imported successfully!")

In [ ]:
class WashSaleTracker:
    """
    Track and identify wash sale violations.
    """
    
    def __init__(self):
        self.transactions = []
        
    def create_sample_transactions(self):
        """Create sample transaction history with wash sales."""
        
        transactions = [
            # Scenario 1: Clear wash sale
            {'date': '2024-12-01', 'ticker': 'AAPL', 'type': 'BUY', 'shares': 100, 'price': 150},
            {'date': '2024-12-15', 'ticker': 'AAPL', 'type': 'SELL', 'shares': 100, 'price': 140},
            {'date': '2024-12-20', 'ticker': 'AAPL', 'type': 'BUY', 'shares': 100, 'price': 142},
            
            # Scenario 2: Safe swap (SPY → VOO)
            {'date': '2024-11-01', 'ticker': 'SPY', 'type': 'BUY', 'shares': 50, 'price': 450},
            {'date': '2024-12-10', 'ticker': 'SPY', 'type': 'SELL', 'shares': 50, 'price': 420},
            {'date': '2024-12-10', 'ticker': 'VOO', 'type': 'BUY', 'shares': 50, 'price': 420},
            
            # Scenario 3: 31-day wait (no violation)
            {'date': '2024-10-01', 'ticker': 'MSFT', 'type': 'BUY', 'shares': 75, 'price': 400},
            {'date': '2024-12-05', 'ticker': 'MSFT', 'type': 'SELL', 'shares': 75, 'price': 380},
            {'date': '2025-01-06', 'ticker': 'MSFT', 'type': 'BUY', 'shares': 75, 'price': 385},
            
            # Scenario 4: Wash sale in prior 30 days
            {'date': '2024-11-15', 'ticker': 'TSLA', 'type': 'BUY', 'shares': 20, 'price': 250},
            {'date': '2024-12-20', 'ticker': 'TSLA', 'type': 'SELL', 'shares': 20, 'price': 220},
            {'date': '2024-12-01', 'ticker': 'TSLA', 'type': 'BUY', 'shares': 20, 'price': 240},
        ]
        
        df = pd.DataFrame(transactions)
        df['date'] = pd.to_datetime(df['date'])
        df['amount'] = df['shares'] * df['price']
        df = df.sort_values('date').reset_index(drop=True)
        
        return df
    
    def identify_wash_sales(self, transactions):
        """Identify wash sale violations."""
        
        df = transactions.copy()
        wash_sales = []
        
        # Find all sales at a loss
        for idx, sale in df[df['type'] == 'SELL'].iterrows():
            ticker = sale['ticker']
            sale_date = sale['date']
            
            # Find prior purchase to calculate loss
            prior_purchase = df[
                (df['ticker'] == ticker) & 
                (df['type'] == 'BUY') & 
                (df['date'] < sale_date)
            ].iloc[-1] if len(df[
                (df['ticker'] == ticker) & 
                (df['type'] == 'BUY') & 
                (df['date'] < sale_date)
            ]) > 0 else None
            
            if prior_purchase is None:
                continue
                
            # Check if sold at a loss
            if sale['price'] < prior_purchase['price']:
                loss_per_share = prior_purchase['price'] - sale['price']
                total_loss = loss_per_share * sale['shares']
                
                # Check 61-day window for repurchases
                window_start = sale_date - timedelta(days=30)
                window_end = sale_date + timedelta(days=30)
                
                repurchases = df[
                    (df['ticker'] == ticker) & 
                    (df['type'] == 'BUY') & 
                    (df['date'] >= window_start) & 
                    (df['date'] <= window_end) &
                    (df['date'] != prior_purchase['date'])
                ]
                
                if len(repurchases) > 0:
                    for _, repurchase in repurchases.iterrows():
                        days_diff = (repurchase['date'] - sale_date).days
                        
                        wash_sales.append({
                            'ticker': ticker,
                            'purchase_date': prior_purchase['date'],
                            'purchase_price': prior_purchase['price'],
                            'sale_date': sale_date,
                            'sale_price': sale['price'],
                            'repurchase_date': repurchase['date'],
                            'repurchase_price': repurchase['price'],
                            'shares': sale['shares'],
                            'loss_per_share': loss_per_share,
                            'total_disallowed_loss': total_loss,
                            'days_between': days_diff,
                            'adjusted_basis': repurchase['price'] + loss_per_share
                        })
        
        return pd.DataFrame(wash_sales)
    
    def generate_wash_sale_report(self, transactions):
        """Generate comprehensive wash sale report."""
        
        wash_sales = self.identify_wash_sales(transactions)
        
        print("="*80)
        print("WASH SALE VIOLATION REPORT")
        print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*80)
        print()
        
        if len(wash_sales) == 0:
            print("No wash sale violations detected.")
            print("All tax-loss harvesting transactions appear compliant.")
        else:
            print(f"WARNING: {len(wash_sales)} WASH SALE VIOLATION(S) DETECTED")
            print()
            
            for idx, ws in wash_sales.iterrows():
                print(f"Violation #{idx+1}: {ws['ticker']}")
                print(f"  Purchase:        {ws['purchase_date'].strftime('%Y-%m-%d')} @ ${ws['purchase_price']:.2f}")
                print(f"  Sale (at loss):  {ws['sale_date'].strftime('%Y-%m-%d')} @ ${ws['sale_price']:.2f}")
                print(f"  Repurchase:      {ws['repurchase_date'].strftime('%Y-%m-%d')} @ ${ws['repurchase_price']:.2f}")
                print(f"  Days Between:    {ws['days_between']} days {'(BEFORE sale)' if ws['days_between'] < 0 else '(AFTER sale)'}")
                print(f"  Loss/Share:      ${ws['loss_per_share']:.2f}")
                print(f"  Total Loss:      ${ws['total_disallowed_loss']:.2f} (DISALLOWED)")
                print(f"  Adjusted Basis:  ${ws['adjusted_basis']:.2f} (was ${ws['repurchase_price']:.2f})")
                print()
            
            total_disallowed = wash_sales['total_disallowed_loss'].sum()
            print(f"TOTAL DISALLOWED LOSSES: ${total_disallowed:,.2f}")
            print(f"  (These losses are deferred to adjusted basis of replacement shares)")
        
        print("="*80)
        
        return wash_sales

print("WashSaleTracker class defined!")

### Example: Wash Sale Detection

In [ ]:
# Create tracker and sample data
tracker = WashSaleTracker()
transactions = tracker.create_sample_transactions()

print("Sample Transaction History:")
print(transactions)

In [ ]:
# Generate wash sale report
wash_sales = tracker.generate_wash_sale_report(transactions)

## 6. Safe Swap Pairs Database

In [ ]:
def get_safe_swap_pairs():
    """Database of safe ETF swap pairs."""
    
    swaps = [
        # S&P 500
        {'category': 'S&P 500', 'from': 'SPY', 'to': 'VOO', 'expense_from': 0.0945, 'expense_to': 0.03},
        {'category': 'S&P 500', 'from': 'SPY', 'to': 'IVV', 'expense_from': 0.0945, 'expense_to': 0.03},
        {'category': 'S&P 500', 'from': 'VOO', 'to': 'SPY', 'expense_from': 0.03, 'expense_to': 0.0945},
        
        # Total Market
        {'category': 'Total Market', 'from': 'VTI', 'to': 'ITOT', 'expense_from': 0.03, 'expense_to': 0.03},
        {'category': 'Total Market', 'from': 'VTI', 'to': 'SCHB', 'expense_from': 0.03, 'expense_to': 0.03},
        {'category': 'Total Market', 'from': 'ITOT', 'to': 'VTI', 'expense_from': 0.03, 'expense_to': 0.03},
        
        # Nasdaq-100
        {'category': 'Nasdaq-100', 'from': 'QQQ', 'to': 'QQQM', 'expense_from': 0.20, 'expense_to': 0.15},
        {'category': 'Nasdaq-100', 'from': 'QQQM', 'to': 'QQQ', 'expense_from': 0.15, 'expense_to': 0.20},
        
        # International Developed
        {'category': 'Int\'l Developed', 'from': 'VEA', 'to': 'IDEV', 'expense_from': 0.05, 'expense_to': 0.04},
        {'category': 'Int\'l Developed', 'from': 'VEA', 'to': 'SCHF', 'expense_from': 0.05, 'expense_to': 0.06},
        
        # Emerging Markets
        {'category': 'Emerging Markets', 'from': 'VWO', 'to': 'IEMG', 'expense_from': 0.08, 'expense_to': 0.09},
        {'category': 'Emerging Markets', 'from': 'VWO', 'to': 'SCHE', 'expense_from': 0.08, 'expense_to': 0.11},
        
        # Total Bond
        {'category': 'Total Bond', 'from': 'AGG', 'to': 'BND', 'expense_from': 0.03, 'expense_to': 0.03},
        {'category': 'Total Bond', 'from': 'AGG', 'to': 'SCHZ', 'expense_from': 0.03, 'expense_to': 0.04},
    ]
    
    df = pd.DataFrame(swaps)
    df['expense_diff'] = df['expense_to'] - df['expense_from']
    
    return df

swap_pairs = get_safe_swap_pairs()

print("SAFE ETF SWAP PAIRS")
print("="*80)
print()

for category in swap_pairs['category'].unique():
    print(f"{category}:")
    subset = swap_pairs[swap_pairs['category'] == category]
    for _, row in subset.iterrows():
        print(f"  {row['from']} → {row['to']:6s} (Expense: {row['expense_from']:.2f}% → {row['expense_to']:.2f}%)")
    print()

## 7. Wash Sale Avoidance Calculator

In [ ]:
def check_wash_sale_window(sale_date, proposed_purchase_date):
    """
    Check if a proposed purchase date would trigger wash sale.
    
    Parameters:
    -----------
    sale_date : str or datetime
        Date of the sale at a loss
    proposed_purchase_date : str or datetime
        Proposed date to repurchase
    """
    
    sale_date = pd.to_datetime(sale_date)
    proposed_date = pd.to_datetime(proposed_purchase_date)
    
    days_diff = (proposed_date - sale_date).days
    
    print("="*60)
    print("WASH SALE WINDOW CHECK")
    print("="*60)
    print()
    print(f"Sale Date:              {sale_date.strftime('%Y-%m-%d')}")
    print(f"Proposed Purchase Date: {proposed_date.strftime('%Y-%m-%d')}")
    print(f"Days Difference:        {days_diff} days")
    print()
    
    # Calculate safe zone
    safe_date_start = sale_date + timedelta(days=31)
    
    if days_diff <= 30:
        print("⚠️  WARNING: WASH SALE VIOLATION!")
        print(f"   Purchase is within 30-day window after sale.")
        print(f"   Loss will be disallowed.")
        print()
        print(f"✓  SAFE TO PURCHASE ON: {safe_date_start.strftime('%Y-%m-%d')} or later")
        print(f"   (Wait {31 - days_diff} more days)")
        result = 'VIOLATION'
    else:
        print("✓  SAFE: No wash sale violation.")
        print(f"   Purchase is {days_diff} days after sale (>30 days).")
        print(f"   Loss can be claimed.")
        result = 'SAFE'
    
    print("="*60)
    
    return result

# Example 1: Too soon (violation)
check_wash_sale_window('2024-12-15', '2024-12-25')

In [ ]:
# Example 2: Safe (31+ days)
check_wash_sale_window('2024-12-15', '2025-01-16')

## 8. Visualization: Wash Sale Timeline

In [ ]:
def visualize_wash_sale_window(sale_date='2024-12-15'):
    """Visualize the 61-day wash sale window."""
    
    sale_date = pd.to_datetime(sale_date)
    
    # Create timeline
    start_date = sale_date - timedelta(days=35)
    end_date = sale_date + timedelta(days=35)
    dates = pd.date_range(start_date, end_date, freq='D')
    
    # Mark zones
    window_start = sale_date - timedelta(days=30)
    window_end = sale_date + timedelta(days=30)
    safe_start = sale_date + timedelta(days=31)
    
    fig, ax = plt.subplots(figsize=(16, 6))
    
    # Plot wash sale window (danger zone)
    ax.axvspan(window_start, window_end, alpha=0.3, color='red', label='Wash Sale Window (61 days)')
    
    # Plot safe zones
    ax.axvspan(start_date, window_start, alpha=0.2, color='green', label='Safe Zone (Before)')
    ax.axvspan(safe_start, end_date, alpha=0.2, color='green', label='Safe Zone (After 31 days)')
    
    # Mark sale date
    ax.axvline(sale_date, color='black', linewidth=3, linestyle='--', label='Sale Date')
    
    # Mark 30-day boundaries
    ax.axvline(window_start, color='red', linewidth=2, linestyle=':', alpha=0.7)
    ax.axvline(window_end, color='red', linewidth=2, linestyle=':', alpha=0.7)
    ax.axvline(safe_start, color='green', linewidth=2, linestyle=':', alpha=0.7)
    
    # Annotations
    ax.text(sale_date, 0.5, 'SALE\n(Loss)', ha='center', va='center', 
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8), fontsize=12, fontweight='bold')
    
    ax.text(window_start + timedelta(days=15), 0.7, 'DANGER:\n30 days before', 
            ha='center', va='center', fontsize=10, color='darkred')
    
    ax.text(sale_date + timedelta(days=15), 0.7, 'DANGER:\n30 days after', 
            ha='center', va='center', fontsize=10, color='darkred')
    
    ax.text(safe_start + timedelta(days=2), 0.3, 'SAFE TO\nREPURCHASE', 
            ha='left', va='center', fontsize=10, color='darkgreen', fontweight='bold')
    
    # Formatting
    ax.set_ylim(0, 1)
    ax.set_xlim(start_date, end_date)
    ax.set_xlabel('Date', fontsize=12)
    ax.set_title(f'Wash Sale 61-Day Window\nSale Date: {sale_date.strftime("%Y-%m-%d")}', 
                fontsize=14, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Remove y-axis
    ax.set_yticks([])
    
    plt.tight_layout()
    plt.show()
    
    # Print key dates
    print("\nKEY DATES:")
    print(f"  Wash Sale Window Start: {window_start.strftime('%Y-%m-%d')} (30 days before)")
    print(f"  Sale Date:              {sale_date.strftime('%Y-%m-%d')}")
    print(f"  Wash Sale Window End:   {window_end.strftime('%Y-%m-%d')} (30 days after)")
    print(f"  Safe to Repurchase:     {safe_start.strftime('%Y-%m-%d')} (31+ days after)")

visualize_wash_sale_window('2024-12-15')

## Quiz: Test Your Knowledge

In [ ]:
questions = [
    {
        'question': "How many days is the wash sale window?",
        'options': ['A) 30 days', 'B) 60 days', 'C) 61 days', 'D) 90 days'],
        'correct': 'C',
        'explanation': "The wash sale window is 61 days total: 30 days before the sale, the sale day itself, and 30 days after the sale."
    },
    {
        'question': "What happens to a disallowed loss from a wash sale?",
        'options': ['A) Lost forever', 'B) Added to cost basis of replacement shares', 'C) Carried forward as capital loss', 'D) Becomes ordinary loss'],
        'correct': 'B',
        'explanation': "The disallowed loss is added to the cost basis of the replacement shares. The loss is deferred, not lost."
    },
    {
        'question': "Which ETF swap would avoid a wash sale when selling SPY at a loss?",
        'options': ['A) Buy more SPY', 'B) Buy SPY options', 'C) Buy VOO (Vanguard S&P 500)', 'D) Buy SPYG (SPY Growth)'],
        'correct': 'C',
        'explanation': "VOO is a different S&P 500 ETF from a different provider. It's not considered substantially identical to SPY, making it a safe swap."
    },
    {
        'question': "You sell AAPL at a loss on Dec 15. When is the earliest safe date to repurchase AAPL?",
        'options': ['A) Dec 16', 'B) Dec 31', 'C) Jan 14', 'D) Jan 15'],
        'correct': 'D',
        'explanation': "You must wait 31 days after the sale. Dec 15 + 31 days = Jan 15."
    },
    {
        'question': "You sell 100 shares of VTI at a $1,000 loss and immediately buy ITOT. Is this a wash sale?",
        'options': ['A) Yes, same market exposure', 'B) Yes, both track total market', 'C) No, different fund companies', 'D) Depends on IRS ruling'],
        'correct': 'C',
        'explanation': "VTI and ITOT are from different providers (Vanguard vs iShares) and are not considered substantially identical despite tracking the same index. This is a legal swap."
    },
    {
        'question': "Can you trigger a wash sale by purchasing the stock BEFORE selling it at a loss?",
        'options': ['A) No, only after matters', 'B) Yes, within 30 days before', 'C) Yes, within 15 days before', 'D) Only if you buy call options'],
        'correct': 'B',
        'explanation': "The wash sale window includes 30 days BEFORE the sale. Buying the stock within 30 days before selling at a loss triggers the wash sale rule."
    },
    {
        'question': "You bought TSLA at $200, sold at $180 (loss), then rebought at $175. What's your adjusted basis?",
        'options': ['A) $175', 'B) $180', 'C) $195', 'D) $200'],
        'correct': 'C',
        'explanation': "Adjusted basis = Repurchase price + Disallowed loss = $175 + ($200 - $180) = $175 + $20 = $195."
    },
    {
        'question': "Which is NOT a safe wash sale avoidance strategy?",
        'options': ['A) Wait 31 days to repurchase', 'B) Swap to similar but different ETF', 'C) Buy call options on the same stock', 'D) Use sector ETF as temporary substitute'],
        'correct': 'C',
        'explanation': "Call options on the same stock are considered substantially identical and will trigger a wash sale. Options don't provide a safe workaround."
    },
    {
        'question': "Your spouse buys the same stock you just sold at a loss in their IRA. Is this a wash sale?",
        'options': ['A) No, different accounts', 'B) No, IRA is tax-advantaged', 'C) Yes, applies to spouse accounts', 'D) Only if in same household'],
        'correct': 'C',
        'explanation': "Wash sale rules apply across accounts you control, including your spouse's accounts and IRAs. This is a wash sale."
    },
    {
        'question': "What's the main benefit of using ETF swaps instead of waiting 31 days?",
        'options': ['A) Lower taxes', 'B) Maintain market exposure', 'C) Higher returns', 'D) Avoid reporting'],
        'correct': 'B',
        'explanation': "ETF swaps let you maintain similar market exposure immediately, avoiding the risk that the stock recovers during a 31-day wait period."
    }
]

print("QUIZ: Wash Sale Rules")
print("="*60)
print(f"Total Questions: {len(questions)}\n")

for i, q in enumerate(questions, 1):
    print(f"Question {i}: {q['question']}")
    for option in q['options']:
        print(f"  {option}")
    print(f"\nCorrect Answer: {q['correct']}")
    print(f"Explanation: {q['explanation']}")
    print("-" * 60)
    print()

## Summary

### Key Takeaways

1. **Wash Sale Rule Basics**
   - 61-day window: 30 days before, sale day, 30 days after
   - Applies when you sell at a loss and repurchase same/substantially identical security
   - Loss is disallowed but added to cost basis of replacement shares
   - Loss is deferred, not permanently lost

2. **Substantially Identical Securities**
   - Same stock ticker: YES (AAPL → AAPL)
   - Same ETF ticker: YES (SPY → SPY)
   - Different S&P 500 ETFs: NO (SPY → VOO is safe)
   - Options on same stock: YES (triggers wash sale)
   - Spouse's accounts: YES (wash sale applies)

3. **Cost Basis Adjustment**
   ```
   New Cost Basis = Repurchase Price + Disallowed Loss
   Holding Period = Combines old and new holding periods
   ```

4. **Legal Avoidance Strategies**
   - **Strategy 1**: Wait 31 days (simple but market risk)
   - **Strategy 2**: Swap to similar ETF (maintains exposure)
   - **Strategy 3**: Use sector ETF as temporary substitute
   - **Best Practice**: Know safe swap pairs for your holdings

5. **Safe ETF Swap Pairs**
   - SPY ↔ VOO / IVV (S&P 500)
   - VTI ↔ ITOT / SCHB (Total Market)
   - QQQ ↔ QQQM (Nasdaq-100)
   - VEA ↔ IDEV / SCHF (International)
   - AGG ↔ BND / SCHZ (Bonds)

### Next Steps

In Day 13, we'll learn about **Asset Location** - the strategic placement of different asset types across taxable and tax-advantaged accounts to minimize total tax drag. We'll discover which investments belong in which account types.

### Additional Resources

- IRS Publication 550 (Wash Sales section)
- IRS Form 8949 instructions
- Bogleheads Wiki: Tax-Loss Harvesting
- ETF Database: ETF Comparison Tools

---

**Disclaimer**: This educational content is for informational purposes only and does not constitute tax advice. Consult a qualified tax professional for personalized guidance.